In [123]:
from string import ascii_letters
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import random
import time
import json

import matplotlib.pyplot as plt
import networkx as nx

import pickle

import cx_Oracle

random.seed(1)

def shift_function(df, groupby_col, columns, shift_value):
    for col in columns:
        df[str(col) + '_shift_' + str(shift_value)] = df.groupby([groupby_col])[col].shift(shift_value)
        
    return df

In [124]:
def add_node_func(name, array, data):
    if name not in array:
        array[name] = {'name':name, 'data':data}
    return array

In [139]:
flag_all_communication = False

In [140]:
buckets_array_complicated = ['root//root',
 '0//0',
 '1-3//1-3',
 '11-30//11-30',
 '31-60//31-60',
 '4-10//4-10',
 '61-90//61-90',
 '91-120//91-120',
 '120+//120+',
 'end//end']

In [141]:
def get_markov_chain_with_break_json(df_value_counts_chain, chain_cut_value, buckets_array_complicated):

    count_all_users_chain = 0
    count_users_chain_cycle = 0
    edge_labels = {}
    nodes_array = {}
    nodes_for_json = []
    edges_for_json = []
    for index, row in df_value_counts_chain.iterrows():
        if row['count_chain'] > chain_cut_value or row['end'] in buckets_array_complicated:
            weight_str = str(row['count_chain']) + "(" + str(round(row['percent'],2)) + "%)"
            edges_for_json_temp = {}
            edges_for_json_temp['from'] = row['start']
            backet = row['start'].split('//')[1]
            backet = backet + "//" + backet

            if row['end'] in buckets_array_complicated:            
                end_name = row['start'] + " -> " + row['end']
                edges_for_json_temp['to'] = end_name
                if row['start'] in buckets_array_complicated:
                    nodes_array = add_node_func(row['start'], nodes_array, {'backet':backet,'color':'orange','font':{'color':'white'}})
                else:
                    nodes_array = add_node_func(row['start'], nodes_array, {'backet':backet,'color':'blue','font':{'color':'white'}})
                nodes_array = add_node_func(end_name, nodes_array, {'backet':backet,'color':'green','font':{'color':'white'}})
            else:
                edges_for_json_temp['to'] = row['end']
                if row['start'] in buckets_array_complicated:
                    nodes_array = add_node_func(row['start'], nodes_array, {'backet':backet,'color':'orange','font':{'color':'white'}})
                else:
                    nodes_array = add_node_func(row['start'], nodes_array, {'backet':backet,'color':'blue','font':{'color':'white'}})
                nodes_array = add_node_func(row['end'], nodes_array, {'backet':backet,'color':'blue','font':{'color':'white'}})

            edges_for_json_temp['label'] = weight_str
            edges_for_json_temp['value'] = row['count_chain']
            edges_for_json_temp['mean'] = row['mean_chain']
            edges_for_json_temp['arrows'] = "to"
            #edges_for_json_temp['physics'] = "false"
            edges_for_json.append(edges_for_json_temp)
            
            if row['start'] != row['end']:
                count_all_users_chain += row['count_chain']
            else:
                count_users_chain_cycle += row['count_chain']    
                
                
                
    print('before cut = ',df_value_counts_chain['count_chain'].sum())
    print('after = ',count_all_users_chain)
    print('cycle = ',count_users_chain_cycle)
    print('cut = ',df_value_counts_chain['count_chain'].sum() - count_all_users_chain - count_users_chain_cycle)
    print('percent cut = ',(df_value_counts_chain['count_chain'].sum() - count_all_users_chain - count_users_chain_cycle) * 100/df_value_counts_chain['count_chain'].sum())
    
#     print('count node:')
    
#     for i in nodes_array:
#         print(i)
        
    print('count edges = ', len(edges_for_json))
    
    #print(nodes_array)
    
    nodes_for_json = []
    for n in nodes_array:
        nodes_for_json_temp = {}
        nodes_for_json_temp['id'] = n
        nodes_for_json_temp['label'] = n
        nodes_for_json_temp['bucket'] = nodes_array[n]['data']['backet']
        nodes_for_json_temp['color'] = nodes_array[n]['data']['color']
        nodes_for_json_temp['font'] = nodes_array[n]['data']['font']
        #data = n[1]
        nodes_for_json.append(nodes_for_json_temp)
    
        
    return nodes_for_json, edges_for_json

def get_graph_from_df(df_actions_merge, start_name, end_name, backet_name, count_name, mean_name):
    #df_value_counts_chain = df_actions_merge.groupby([end_name,start_name])[count_name].count().reset_index()
    df_value_counts_chain = df_actions_merge.groupby([start_name,end_name, backet_name]).agg({mean_name:'mean',count_name:'count'}).reset_index()
    df_value_counts_chain = df_value_counts_chain.rename(columns={end_name:"end", start_name:"start", count_name:"count", mean_name:"mean", backet_name:"backet"})
    df_value_counts_chain = df_value_counts_chain.sort_values(by=['count'], ascending=False)
    df_count_start_in_node = df_value_counts_chain.groupby(['start']).sum().reset_index().sort_values(by='count', ascending=False)
    df_value_counts_chain = df_value_counts_chain.merge(df_count_start_in_node, on='start', suffixes=['_chain','_all_in_node'])
    df_value_counts_chain['percent'] = (df_value_counts_chain['count_chain'] * 100)/df_value_counts_chain['count_all_in_node']
    df_value_counts_chain['cumulative_sum'] = df_value_counts_chain.groupby('start')['percent'].cumsum()
    
    df_value_counts_chain['backet_start'] = df_value_counts_chain['start'].apply(lambda x: x.split('//')[1])
    
    return df_value_counts_chain, df_count_start_in_node

In [142]:
with open("../data_retrieval/dict_of_vocabulary.json", "r") as read_file:
    dict_of_vocabulary = json.load(read_file)

if not flag_all_communication:
    df_actions_merge_with_info = pd.read_csv('../data_retrieval/df_actions_merge__part_communication.csv', index_col=0,sep=';')
    add_str = "part"
else:
    df_actions_merge_with_info = pd.read_csv('../data_retrieval/df_actions_merge__all_communication.csv', index_col=0,sep=';')
    add_str = "all"

df_value_counts_chain, df_count_start_in_node = get_graph_from_df(df_actions_merge_with_info, 'MARCOV_TARGET_COLUMN_shift_1', 'MARCOV_TARGET_COLUMN','MARCOV_TARGET_COLUMN_BATCHES', 'SYSID_DOG', 'MARCOV_TARGET_ROW_NUMBER_IN_BUCKET')
nodes_for_json, edges_for_json = get_markov_chain_with_break_json(df_value_counts_chain, 0, buckets_array_complicated)
with open('graphs/data_'+add_str+'.json','w', encoding='utf-8') as f:
    f.write(json.dumps({"nodes":nodes_for_json, "edges":edges_for_json}))
    
# кн
print("\n КН")
df_value_counts_chain, df_count_start_in_node = get_graph_from_df(df_actions_merge_with_info[df_actions_merge_with_info['PRODUCT_ID_label_encoder'] == dict_of_vocabulary['PRODUCT_ID']['КН']], 'MARCOV_TARGET_COLUMN_shift_1', 'MARCOV_TARGET_COLUMN','MARCOV_TARGET_COLUMN_BATCHES', 'SYSID_DOG', 'MARCOV_TARGET_ROW_NUMBER_IN_BUCKET')
nodes_for_json, edges_for_json = get_markov_chain_with_break_json(df_value_counts_chain, 0, buckets_array_complicated)
with open('graphs/data_kn_'+add_str+'.json','w', encoding='utf-8') as f:
    f.write(json.dumps({"nodes":nodes_for_json, "edges":edges_for_json}))
    
print("\n авто")
df_value_counts_chain, df_count_start_in_node = get_graph_from_df(df_actions_merge_with_info[df_actions_merge_with_info['PRODUCT_ID_label_encoder'] != dict_of_vocabulary['PRODUCT_ID']['КН']], 'MARCOV_TARGET_COLUMN_shift_1', 'MARCOV_TARGET_COLUMN','MARCOV_TARGET_COLUMN_BATCHES', 'SYSID_DOG', 'MARCOV_TARGET_ROW_NUMBER_IN_BUCKET')
nodes_for_json, edges_for_json = get_markov_chain_with_break_json(df_value_counts_chain, 0, buckets_array_complicated)
with open('graphs/data_auto_'+add_str+'.json','w', encoding='utf-8') as f:
    f.write(json.dumps({"nodes":nodes_for_json, "edges":edges_for_json}))

/home/server/anaconda3/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


before cut =  1039376
after =  773938
cycle =  265438
cut =  0
percent cut =  0.0
count edges =  394

 КН
before cut =  434311
after =  312067
cycle =  122244
cut =  0
percent cut =  0.0
count edges =  328

 авто
before cut =  605065
after =  461871
cycle =  143194
cut =  0
percent cut =  0.0
count edges =  373


In [143]:
df_actions_merge_with_info['ACTION_DATE'] = pd.to_datetime(df_actions_merge_with_info['ACTION_DATE'])
df_months = pd.DataFrame(df_actions_merge_with_info['ACTION_DATE'].dt.strftime('%Y-%m').unique())
df_months[0] = pd.to_datetime(df_months[0])
df_months[1] = df_months[0] + pd.DateOffset(months=1)
df_months[0] = df_months[0].dt.strftime('%Y-%m')
df_months[1] = df_months[1].dt.strftime('%Y-%m')

for i, m in df_months.iterrows():
    if (m[0] == '1800-01') or (m[0] == '2200-01'):
        continue
    print(m[0],m[1])
    df_temp = df_actions_merge_with_info[((df_actions_merge_with_info['MARCOV_TARGET_COLUMN'] == "root//root")\
                              & (df_actions_merge_with_info['MARCOV_TARGET_COLUMN'] == "end//end")) \
                           | ((df_actions_merge_with_info['ACTION_DATE'] > m[0]) \
                              & (df_actions_merge_with_info['ACTION_DATE'] < m[1]))]
    
    df_value_counts_chain, df_count_start_in_node = get_graph_from_df(df_temp, 'MARCOV_TARGET_COLUMN_shift_1', 'MARCOV_TARGET_COLUMN','MARCOV_TARGET_COLUMN_BATCHES', 'SYSID_DOG', 'MARCOV_TARGET_ROW_NUMBER_IN_BUCKET')
    nodes_for_json, edges_for_json = get_markov_chain_with_break_json(df_value_counts_chain, 0, buckets_array_complicated)
    with open('graphs/months/'+add_str+'/data_'+add_str+'_'+str(m[0])+'_'+str(m[1])+'.json','w', encoding='utf-8') as f:
        f.write(json.dumps({"nodes":nodes_for_json, "edges":edges_for_json}))

    # кн
    print("\n КН")
    df_value_counts_chain, df_count_start_in_node = get_graph_from_df(df_temp[df_temp['PRODUCT_ID_label_encoder'] == dict_of_vocabulary['PRODUCT_ID']['КН']], 'MARCOV_TARGET_COLUMN_shift_1', 'MARCOV_TARGET_COLUMN','MARCOV_TARGET_COLUMN_BATCHES', 'SYSID_DOG', 'MARCOV_TARGET_ROW_NUMBER_IN_BUCKET')
    nodes_for_json, edges_for_json = get_markov_chain_with_break_json(df_value_counts_chain, 0, buckets_array_complicated)
    with open('graphs/months/'+add_str+'/data_kn_'+add_str+'_'+str(m[0])+'_'+str(m[1])+'.json','w', encoding='utf-8') as f:
        f.write(json.dumps({"nodes":nodes_for_json, "edges":edges_for_json}))

    print("\n авто")
    df_value_counts_chain, df_count_start_in_node = get_graph_from_df(df_temp[df_temp['PRODUCT_ID_label_encoder'] != dict_of_vocabulary['PRODUCT_ID']['КН']], 'MARCOV_TARGET_COLUMN_shift_1', 'MARCOV_TARGET_COLUMN','MARCOV_TARGET_COLUMN_BATCHES', 'SYSID_DOG', 'MARCOV_TARGET_ROW_NUMBER_IN_BUCKET')
    nodes_for_json, edges_for_json = get_markov_chain_with_break_json(df_value_counts_chain, 0, buckets_array_complicated)
    with open('graphs/months/'+add_str+'/data_auto_'+add_str+'_'+str(m[0])+'_'+str(m[1])+'.json','w', encoding='utf-8') as f:
        f.write(json.dumps({"nodes":nodes_for_json, "edges":edges_for_json}))

2020-01 2020-02
before cut =  3146
after =  3143
cycle =  3
cut =  0
percent cut =  0.0
count edges =  4

 КН
before cut =  1756
after =  1756
cycle =  0
cut =  0
percent cut =  0.0
count edges =  3

 авто
before cut =  1390
after =  1387
cycle =  3
cut =  0
percent cut =  0.0
count edges =  3
2020-02 2020-03
before cut =  6954
after =  4931
cycle =  2023
cut =  0
percent cut =  0.0
count edges =  36

 КН
before cut =  3569
after =  2479
cycle =  1090
cut =  0
percent cut =  0.0
count edges =  32

 авто
before cut =  3385
after =  2452
cycle =  933
cut =  0
percent cut =  0.0
count edges =  33
2020-03 2020-04
before cut =  13654
after =  8438
cycle =  5216
cut =  0
percent cut =  0.0
count edges =  55

 КН
before cut =  7103
after =  4229
cycle =  2874
cut =  0
percent cut =  0.0
count edges =  51

 авто
before cut =  6551
after =  4209
cycle =  2342
cut =  0
percent cut =  0.0
count edges =  50
2020-04 2020-05
before cut =  14267
after =  2172
cycle =  12095
cut =  0
percent cut =  0.

In [144]:
#df_actions_merge_with_info[df_actions_merge_with_info['ACTION_DATE'] == '2021-06-02 00:00:00']